In [15]:

#Working Copy  2022-02-25 0900AM
#pip install BeautifulSoup as b  & #pip install requests

from bs4 import BeautifulSoup as b
import numpy as np
import requests as r
import pandas as p
import math as m
from datetime import date, timedelta, datetime

# getting search days from user
def getDates(txt):
    msg1 = 'Enter the search ' + txt + ' in YYYY-MM-DD format'
    startDate = input(msg1)
    gDate = 0 

    while gDate != 1:
        try:
            Syear, Smonth, Sday = map(int, startDate.split('-'))
            date1 = date(Syear, Smonth, Sday)
        except: 
            startDate = input(msg1)
            continue
        gDate = 1 
    return date1
date1 = getDates('startDate')
date2 = getDates('endDate')

#Get new search string for each day
base = date1
stringA = "https://www.fpds.gov/ezsearch/fpdsportal?q=+SIGNED_DATE%3A%5B"
base = date1
endt = base + timedelta(days=1)
stringB = str(base.year)+"%2F"+base.strftime("%m") + "%2F" + base.strftime("%d") + "%2C"+str(endt.year)+"%2F" + endt.strftime("%m") + "%2F" +endt.strftime("%d") + "%5D&s=FPDS.GOV&templateName=1.5.2&indexName=awardfull&x=30&start="
stringC = '0'
myURLsearch = (stringA + stringB + stringC)
myURLsearch
downloadURL = r.get(myURLsearch,headers={})
soup = b(downloadURL.text,"html.parser")
#CSS
fulltable_select  = soup.select('span.results_heading')
myLen = m.ceil(int(fulltable_select[1].find_all('b')[2].text)/30)
print(myLen)

runInt = 1
runInt = int(runInt)
while base <= date2:
    endt = base + timedelta(days=1)
    stringB = str(base.year)+"%2F"+base.strftime("%m") + "%2F" + base.strftime("%d") + "%2C"+str(endt.year)+"%2F" + endt.strftime("%m") + "%2F" +endt.strftime("%d") + "%5D&s=FPDS.GOV&templateName=1.5.2&indexName=awardfull&x=24&y=7&start="
    stringC = int(stringC) + 30
    stringC = str(stringC)
    myURLsearch = (stringA + stringB + stringC)
    base = base + timedelta(days=1)
    
    
    if runInt < myLen:
        wID = runInt
        def wrkHoss(wID):
            downloadURL = r.get(myURLsearch,headers={})
            soup = b(downloadURL.text,"html.parser")
            fulltable_select  = soup.select('span.results_text')
            datalist = []
            slop =[]
            for d in fulltable_select: 
                try: 
                    datalist.append(d.a.contents)
                except:
                    try:
                        datalist.append(d.span.contents)
                    except:
                        #formerly known as slop
                        datalist.append(str(d.contents).replace("\\t","").replace("\\n","").replace("[","").replace("]","").replace("\'",""))
                        continue
            newlist = ["".join(d) if type(d) is list else d for d in datalist ]
            dlarr = np.array(newlist).reshape(30,19)
            #[",".join(d) if type(d) is list else   arr in dlarr]
            df = p.DataFrame(data= dlarr
            , columns=['AwardID','AwardType','LegalBusinessName','ContractingAgency','DateSigned','ActionObligation','ReferencedIDV','ContractingOffice','NAICS_Code','PSC_Code','EntityCity','UniqueEntityID_DUNS', 'EntityState','UniqueEntityID_SAM','Zip','UltimateParentUniqueEntityID_DUNS','UltimateParentLegalBusinessName','UltimateParentUniqueEntityID_SAM','CAGE_Code']
            )
            r2 = datetime.today().strftime("%Y%m%d%H%M%S")
            fileName = 'fpds_ng_' + r2 + '.csv'
            # Create run interval interger
            if wID < 2:
                df.to_csv(fileName,index = False)
                
            else:

                with open(fileName, "a", newline="") as file:
                    writer = csv.writer(file,delimiter= ",",)
                    writer.writerows(dlarr)
                    runInt = runInt + 1
                    print(runInt)


92


In [16]:
%whos

Variable           Type             Data/Info
---------------------------------------------
b                  type             <class 'bs4.BeautifulSoup'>
base               date             2022-02-26
date               type             <class 'datetime.date'>
date1              date             2022-02-24
date2              date             2022-02-25
datetime           type             <class 'datetime.datetime'>
downloadURL        Response         <Response [200]>
endt               date             2022-02-26
fulltable_select   list             n=2
getDates           function         <function getDates at 0x00000115195A08C8>
m                  module           <module 'math' (built-in)>
myLen              int              92
myURLsearch        str              https://www.fpds.gov/ezse<...>ardfull&x=24&y=7&start=60
np                 module           <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
os                 module           <module 'os' from 'C:\\Pr<...>naconda3_